# Preprocessing the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Function for loading the Dataset.

In [ ]:
import numpy as np
from urllib.request import urlopen

def load_fashion_mnist():

    url_base = "https://www.math.unipd.it/~dasan/"
    Y_train = np.frombuffer(urlopen(url_base + "train-labels-idx1-ubyte").read(), dtype=np.uint8, offset=8)
    X_train = np.frombuffer(urlopen(url_base + "train-images-idx3-ubyte").read(), dtype=np.uint8, offset=16).reshape(len(Y_train), 784) 
                                                                                            
    Y_test = np.frombuffer(urlopen(url_base + "t10k-labels-idx1-ubyte").read(), dtype=np.uint8, offset=8)
    X_test = np.frombuffer(urlopen(url_base + "t10k-images-idx3-ubyte").read(), dtype=np.uint8, offset=16).reshape(len(Y_test), 784)

    return X_train, Y_train, X_test, Y_test

X_train_val, Y_train_val, X_test, Y_test = load_fashion_mnist()


First of all let's have a look of the data. The images are already in grey-scale since every pixel has just one number.

In [ ]:
import matplotlib.pyplot as plt 
from collections import Counter

print(X_train_val.shape, Y_train_val.shape, X_test.shape, Y_test.shape) #shape of the data
#value of the data
print(f"Min: {X_train_val[0].min()}")
print(f"Max: {X_train_val[0].max()}")
print(f"Mean: {X_train_val[0].mean():.5f}")
print(f"Min: {X_train_val[10].min()}")
print(f"Max: {X_train_val[10].max()}")
print(f"Mean: {X_train_val[10].mean():.5f}")
#grayscale goes from 0 to 255

#what are our images
labels_name={
    0:'T-shirt/top',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8:'Bag',
    9:'Ankle boot'
}


print(Counter(Y_train_val)) #classes are perfectly balanced

_, axes = plt.subplots(nrows=1, ncols=10, figsize=(30, 9)) #data displayed
for ax, x, label in zip(axes,X_train_val[0:10], Y_train_val[0:10]):
  ax.set_axis_off()
  X_image=x.reshape(28,28)
  ax.imshow(X_image, cmap=plt.cm.gray_r, interpolation="nearest")
  ax.set_title("Training: %s" % labels_name[label])

Data are perfectly balanced, so far as I know I won't need Under or Over Sampling methods.

I build the validation and the training set.

In [ ]:
from sklearn.model_selection import train_test_split
# we prefer a small validation set since dataset is quite big
X_train, X_val, y_train, y_val = train_test_split(X_train_val, Y_train_val, test_size=0.1, shuffle=False)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
print(Counter(y_train))
#data are still balanced

Now I build several type of training set and validation set:
- two set preprocessed by MinMax scaler and Standard Scaler,
- one set with a smaller size for classifiers that do not run in a reasonable time with the whole batch of data,

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#training and validation sets rescaled in 0-1 values 
minmax_scal=MinMaxScaler().fit(X_train_val)
X_train_mm=minmax_scal.transform(X_train)
X_val_mm=minmax_scal.transform(X_val)
# this means X_train_mm=X_train/255

#training and validation sets rescaled with 0 mean 1 std
stand_scal=StandardScaler().fit(X_train_val)
X_train_ss=stand_scal.transform(X_train)
X_val_ss=stand_scal.transform(X_val)


In [ ]:
#smaller data sets
X_train_small, X_val_small, y_train_small, y_val_small = train_test_split(X_val, y_val, test_size=0.1, shuffle=False)
print(X_train_small.shape, y_train_small.shape, X_val_small.shape, y_val_small.shape)
print(Counter(y_train_small))

#data more or less balanced

Let's start to implement the classifiers.

For this classification task there's no reason to prefer a false positive to a false negative. We evaluate the performance of all the classifier by the accuracy on the validation set prediction.
The best accuracy score on validation set for classifier is reported in the title of each paragraph.

# 2) Decision Tree and Random Forest. (0.88250)

I first run a Grid Search with a tree classifier trained on the small dataset.
Than, knowing the best parameters, I fit a tree classifier on the whole batch of data.

In [ ]:
#runtime=8-9 minutes
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import tree
 

#parameters
param_grid_test = {
    'max_depth': [5,10,50,None],
    'min_samples_split': [2,8,20],
    'min_samples_leaf': [1,4,8]
}

#model definition
model = tree.DecisionTreeClassifier(random_state=123,criterion='entropy')

#grid-search object
clf_2 = GridSearchCV(estimator = model, param_grid=param_grid_test, 
                   cv = 5, scoring = "accuracy",verbose=0)

#training
clf_2.fit(X_train_small,y_train_small)

print(clf_2.best_params_)
print(clf_2.best_score_)

# expected output:
# {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}
# 0.757037037037037

In [ ]:
#new model
clf_tree=tree.DecisionTreeClassifier(random_state=123,
                                     criterion='entropy',
                                     max_depth = 10,
                                     min_samples_leaf = 4,
                                     min_samples_split = 2 )

#training on the whole batch of data
clf_tree.fit(X_train,y_train)

#make prediction
y_pred_tree_train=clf_tree.predict(X_train) 
y_pred_tree_val=clf_tree.predict(X_val)
print(f"accuracy score trees train:\t{accuracy_score(y_train,y_pred_tree_train):.5f}, \tval: \t{accuracy_score(y_val,y_pred_tree_val):.5f}")

#expected output:
#accuracy score trees train:	0.84770, 	val: 	0.80717

I repeat the same process for a random forest classifier.

In [ ]:
#runtime= 10 min
from sklearn.ensemble import RandomForestClassifier

#parameters
param_grid_test = {
    'n_estimators':[5,10,100],
    'max_depth': [5,10,None],
    'min_samples_split': [2,10],
    'min_samples_leaf': [1,10]
}

#model definition
model = RandomForestClassifier(random_state=123,criterion='entropy')

#grid-search object
clf_3 = GridSearchCV(estimator = model, param_grid=param_grid_test, 
                   cv = 5, scoring = "accuracy",verbose=0)

#training
clf_3.fit(X_train_small,y_train_small)

print(clf_3.best_params_)
print(clf_3.best_score_)

# expected output:
# {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
# 0.8472222222222221
# basically the default parameters ---> percfectly fit the train set

In [ ]:
#new model
clf_forest=RandomForestClassifier(random_state=123,
                                  criterion='entropy',
                                  n_estimators = 100,
                                  max_depth = None ,
                                  min_samples_split =  2,
                                  min_samples_leaf =  1
                                  )

#training
clf_forest.fit(X_train,y_train)

#make prediction
y_pred_forest_train=clf_forest.predict(X_train)
y_pred_forest_val=clf_forest.predict(X_val)
print(f"accuracy score trees train:\t{accuracy_score(y_train,y_pred_forest_train):.5f}, \tval: \t{accuracy_score(y_val,y_pred_forest_val):.5f}")

#output
#F1 score trees train:      	1.00000, 	val: 	0.87996
#accuracy score trees train:	1.00000, 	val: 	0.88233

Let's have a look of the confusion matrices. Then I can visualize what tipe of samples are misclassified to understand how much big is the error.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay 

cm_tree = confusion_matrix(y_val, y_pred_tree_val, labels=clf_2.classes_)
print('       TREE: confusion matrix')
print()
disp = ConfusionMatrixDisplay(confusion_matrix=cm_tree, display_labels=clf_tree.classes_)
disp.plot()
plt.show()


cm_forest = confusion_matrix(y_val, y_pred_forest_val, labels=clf_forest.classes_)
print('       FOREST: confusion matrix')
print()
disp = ConfusionMatrixDisplay(confusion_matrix=cm_forest, display_labels=clf_forest.classes_)
disp.plot()
plt.show()


It seems that the target labeled with number 6 (Shirt) is often confused for class labeled with number 0 (T-shirt/top). 
If it was the only error it will be quite a good performance.

In [ ]:
indices = np.all([ (y_val[0:600]==6), (y_pred_forest_val[0:600]==0) ], axis=0) 

_, axes = plt.subplots(nrows=1, ncols=sum(indices==True), figsize=(20, 6))
for ax, x, label, pred_label in zip(axes, X_val[0:600][indices], y_val[0:600][indices], y_pred_forest_val[0:600][indices]):
  ax.set_axis_off()
  ax.imshow(x.reshape(28,28), cmap=plt.cm.gray_r, interpolation="nearest")
  ax.set_title("Test: %s\nPred: %s" % (labels_name[label], labels_name[pred_label]))

#scambia facilmente i vestitini per magliette

## What happen with scaled data?

The best accuracy score on validation set is reached using minmax scaler and a random forest classifier.
Actually there are not real improvements in using Standard scaled datasets or hog preprocessed data, uncomment other lines to see what happen. 

In [ ]:
#minmax scaler

"""
#tree

clf_2_mm=tree.DecisionTreeClassifier(random_state=123,
                                     criterion='entropy',
                                     max_depth = 10,
                                     min_samples_leaf = 4,
                                     min_samples_split = 2 ) 

#training
clf_2_mm.fit(X_train_mm,y_train)

#prediction
y_pred_tree_train_mm=clf_2_mm.predict(X_train_mm)
y_pred_tree_val_mm=clf_2_mm.predict(X_val_mm)
print(f"Decision Tree with minmax scaler data")
print(f"accuracy score trees train:\t{accuracy_score(y_train,y_pred_tree_train_mm):.5f}, \tval: \t{accuracy_score(y_val,y_pred_tree_val_mm):.5f}")
"""

#forest
clf_3_mm=RandomForestClassifier(random_state=123,
                                  criterion='entropy',
                                  n_estimators = 100,
                                  max_depth = None ,
                                  min_samples_split =  2,
                                  min_samples_leaf =  1
                                  )

#training
clf_3_mm.fit(X_train_mm,y_train)

#prediction
y_pred_forest_train_mm=clf_3_mm.predict(X_train_mm)
y_pred_forest_val_mm=clf_3_mm.predict(X_val_mm)
print(f"Random Forest with minmax scaler data")
print(f"accuracy score trees train:\t{accuracy_score(y_train,y_pred_forest_train_mm):.5f}, \tval: \t{accuracy_score(y_val,y_pred_forest_val_mm):.5f}")

#expected output:
#Decision Tree with minmax scaler data
#accuracy score trees train:	0.84770, 	val: 	0.80717
#Random Forest with minmax scaler data
#accuracy score trees train:	1.00000, 	val: 	0.88250# best for forest

In [ ]:
#standard scaler

"""
#tree
clf_2_ss=tree.DecisionTreeClassifier(random_state=123,
                                     criterion='entropy',
                                     max_depth = 10,
                                     min_samples_leaf = 4,
                                     min_samples_split = 2 ) 

#training
clf_2_ss.fit(X_train_ss,y_train)

#prediction
y_pred_tree_train_ss=clf_2_ss.predict(X_train_ss) 
y_pred_tree_val_ss=clf_2_ss.predict(X_val_ss)
print(f"Decision Tree with Standard scaler data")
print(f"accuracy score trees train:\t{accuracy_score(y_train,y_pred_tree_train_ss):.5f}, \tval: \t{accuracy_score(y_val,y_pred_tree_val_ss):.5f}")


#forest
clf_3_ss=RandomForestClassifier(random_state=123,
                                  criterion='entropy',
                                  n_estimators = 100,
                                  max_depth = None ,
                                  min_samples_split =  2,
                                  min_samples_leaf =  1
                                  )

#training
clf_3_ss.fit(X_train_ss,y_train)

#prediction
y_pred_forest_train_ss=clf_3_ss.predict(X_train_ss)
y_pred_forest_val_ss=clf_3_ss.predict(X_val_ss)
print(f"Random Forest with Standard scaler data")
print(f"accuracy score trees train:\t{accuracy_score(y_train,y_pred_forest_train_ss):.5f}, \tval: \t{accuracy_score(y_val,y_pred_forest_val_ss):.5f}")
"""

#expected output:
#Decision Tree with Standard scaler data
#accuracy score trees train:	0.84770, 	val: 	0.80700
#Random Forest with Standard scaler data
#accuracy score trees train:	1.00000, 	val: 	0.88233

In [ ]:
#hog transformation

"""
#tree
clf_2_hog=tree.DecisionTreeClassifier(random_state=123,
                                     criterion='entropy',
                                     max_depth = 10,
                                     min_samples_leaf = 4,
                                     min_samples_split = 2 ) 

#training
clf_2_hog.fit(X_train_hog,y_train_small)

#prediction
y_pred_tree_train_hog=clf_2_hog.predict(X_train_hog) 
y_pred_tree_val_hog=clf_2_hog.predict(X_val_hog)
print(f"Decision Tree with hog transformed data")
print(f"accuracy score trees train:\t{accuracy_score(y_train_small,y_pred_tree_train_hog):.5f}, \tval: \t{accuracy_score(y_val_small,y_pred_tree_val_hog):.5f}")


#forest
clf_3_hog=RandomForestClassifier(random_state=123,
                                  criterion='entropy',
                                  n_estimators = 100,
                                  max_depth = None ,
                                  min_samples_split =  2,
                                  min_samples_leaf =  1
                                  )

#training
clf_3_hog.fit(X_train_hog,y_train_small)

#prediction
y_pred_forest_train_hog=clf_3_hog.predict(X_train_small)
y_pred_forest_val_hog=clf_3_hog.predict(X_val_small)
print(f"Random Forest with hog transformed data")
print(f"accuracy score trees train:\t{accuracy_score(y_train_small,y_pred_forest_train_hog):.5f}, \tval: \t{accuracy_score(y_val_small,y_pred_forest_val_hog):.5f}")
"""

#expected output:
#Decision Tree with hog transformed data
#accuracy score trees train:	0.90815, 	val: 	0.77167
#Random Forest with hog transformed data
#accuracy score trees train:	0.36852, 	val: 	0.37500



# 3) Support Vector Machine. (0.89350)

SVM can run only with small data set, training on the all data takes 3 minutes and make a prediction on the training set takes 14 minutes. For this reason I prefer to avoid to make a prediction for the training set, computing the accuracy score only for the validation set. (Uncomment the cell of the training set to see its accuracy score)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm

#parameters
svc_grid_params = {
    'C': ( 1, 10, 100),
    'kernel': ('rbf', 'linear'),
}

#grid search object
model = svm.SVC(random_state = 123)
clf_4 = GridSearchCV(estimator = model,
                     param_grid = svc_grid_params, 
                     scoring='accuracy',
                     n_jobs= -1, cv = 5, verbose = 1)

#training 
clf_4.fit(X_train_small,y_train_small)


print(clf_4.best_params_)
print(clf_4.best_score_)

#expected output:
#Fitting 5 folds for each of 6 candidates, totalling 30 fits
#{'C': 100, 'kernel': 'rbf'}
#0.8585185185185186

In [ ]:
#model definition with best parameters
clf_svm=svm.SVC(random_state = 123,
                kernel = 'rbf',
                C=100)

#training 
clf_svm.fit(X_train,y_train)

#make a prediction for the validation set
y_pred_svm_val=clf_svm.predict(X_val) 
print(f"accuracy score SVM on validation set: \t{accuracy_score(y_val,y_pred_svm_val):.5f}")

#expected output:
#accuracy score SVM on validation set: 	0.89183

In [ ]:
"""
y_pred_svm_train=clf_svm.predict(X_train) 
print(f"accuracy score SVM on trainin set: \t{accuracy_score(y_train,y_pred_svm_train):.5f}")
"""

#expected output
#accuracy score SVM on trainin set: 	0.99969

## What happen with scaled data?

The best accuracy score is reached using the Standard Scaled data and is not so better than the accuracy score just gained. 
Uncomment the other lines to see what happen with MinMax scaler and hog transformation.

In [ ]:
#minmax scaler

"""
#model
clf_svm_mm=svm.SVC(random_state = 123,
                kernel = 'rbf',
                C=100)

#training 
clf_svm_mm.fit(X_train_mm,y_train)

#make a prediction for the validation set
y_pred_svm_val=clf_svm_mm.predict(X_val_mm) 
print(f"accuracy score SVM  with MinMax scaler data on val: \t{accuracy_score(y_val,y_pred_svm_val):.5f}")
"""

#Standard scaler

#model
clf_svm_ss=svm.SVC(random_state = 123,
                kernel = 'rbf',
                C=100)

#training 
clf_svm_ss.fit(X_train_ss,y_train)

#make a prediction for the validation set
y_pred_svm_val=clf_svm_ss.predict(X_val_ss) 
print(f"accuracy score SVM  with Standardscaler data on val: \t{accuracy_score(y_val,y_pred_svm_val):.5f}")


"""
#Hog transformed data

#model
clf_svm_hog=svm.SVC(random_state = 123,
                kernel = 'rbf',
                C=100)

#training 
clf_svm_hog.fit(X_train_hog,y_train_small)

#We can make prediction on both train and val because we are using small data set
y_pred_svm_train=clf_svm_hog.predict(X_train_hog)
y_pred_svm_val=clf_svm_hog.predict(X_val_hog) 
print(f"accuracy score SVM  with Hog transformed data on train: \t{accuracy_score(y_train_small,y_pred_svm_train):.5f}, val: \t{accuracy_score(y_val_small,y_pred_svm_val):.5f}")
"""


#questi score sono falsati perchè il validation è calcolato su un set che 
#expected output:
#accuracy score SVM  with MinMax scaler data on val: 	0.89167
#accuracy score SVM  with Standardscaler data on val: 	0.89350
#accuracy score SVM  with Hog trnasformed data on train: 	1.00000, val: 	0.84333

# 4) Artificial Neural Network. (0.89417)

In Keras documentation we can read that "images need to be read and decoded into integer tensors, then converted to floating point and normalized to small values (usually between 0 and 1)".

For this reason in the ANN implementation I use only data preprocessed by MinMax scaler and Standard Scaler.

# First of all I implement a function that allows me to make e grid search for the optimal parameters.

In [ ]:
#libraries I need
import tensorflow
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.utils import to_categorical 
from keras.utils.vis_utils import plot_model
from tensorflow.python.framework.random_seed import set_random_seed
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [ ]:
#function for define and train more NN
def MLP(X,y,vector_units,n_epochs,stop,n_patience):
    #X already scaled
    #y not transformed by to_categorical
    
    #processing data
    features=np.shape(X)[1] #the input size
    num_classes=len(set(y))
    np.random.seed(123)
    set_random_seed(2)

    model=Sequential()
    n_layers=len(vector_units) #we consider only the hidden layer
    if n_layers==0:
      model.add(Dense(input_dim=features, 
                      units = num_classes, 
                      activation= 'softmax'))
    else:
      for i in range(n_layers):
        if i==0:
          model.add(Dense(input_dim=features,
                          units=vector_units[0],
                          activation='relu'))
        else: 
          model.add(Dense(units=vector_units[i],
                          activation='relu'))
      model.add(Dense(units = num_classes, 
                      activation= 'softmax'))
    
    #configure the model
    model.compile(loss='categorical_crossentropy', 
                optimizer='sgd', 
                metrics=['accuracy']) 
    
    #EarlyStopping
    if stop=='True':
      stop_er=EarlyStopping(monitor='val_loss', #quantity to be monitored
                    mode='min', #we look for decreasing patterns stop 
                    patience = n_patience, #number of epochs with no improvements
                    verbose=1)
    else:
      stop_er=None
    #training
    y=to_categorical(y,num_classes)
    train=model.fit(X,y,epochs=n_epochs,
                    batch_size=32, 
                    verbose=0,
                    validation_split=0.1,
                    callbacks=stop_er)
    
    return model,train

I create two smaller data sets of scaled data and the one-hot encoding of the labels.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#smaller data set for the grid search
minmax_scal=MinMaxScaler().fit(X_train_small)
X_train_mms=minmax_scal.transform(X_train_small) 
X_val_mms=minmax_scal.transform(X_val_small) 

#second smaller dataset for a comparison
standard_scal=StandardScaler().fit(X_train_small)
X_train_sss=standard_scal.transform(X_train_small) 
X_val_sss=standard_scal.transform(X_val_small) 

#one hot encoding
y_train_cs=to_categorical(y_train_small,10)
y_val_cs=to_categorical(y_val_small,10)


Now I can train several NN to find the best parameters.
Even though I used smaller data set this cell requires 15 or more minutes to run. Since this is actually the best model among the others it's worth it.

In [ ]:
#parameters of the function

#to avoid a 50 minutes run I put a shortes version of vector_units_list but the results is the same
# vector_units_list=[[784],[784,10],[500,200],[784,100,50],[784,200,50,10]] #long research
vector_units_list=[[784],[784,10]]
n_epochs_list=[100,500]
n_patience_list=[5,15,25]
stop='True'
#let save the maximum accuracy
acc=0
accuracy_test=[]
i=0
#grid-search
for n_epochs in n_epochs_list:
  for vector in vector_units_list:
    for n_patience in n_patience_list:

      #train a NN 
      m,t=MLP(X_train_mms,y_train_small,vector,n_epochs,stop,n_patience)
      print(i)
      i+=1
      #save the greater accuracy
      accuracy_val=m.evaluate(X_val_mms,y_val_cs,verbose=0)[1]
      if accuracy_val>acc:
        acc=accuracy_val
        win=[vector,n_epochs,n_patience]
        model=m
        accuracy_test.append(accuracy_val)

print('\n\n\n')
print(model.summary())
print('best parameters: ',win)
plt.plot(accuracy_test)
plt.title(f"accuracy growing")
plt.ylabel('accuracy')
plt.xlabel('grid search')
plt.legend(['test acc'], loc='upper left')
plt.show()

#expected output:
# best parameters: [[784], 100, 15]

Now I can train on the whole batch of data one Artificial Neural Network with the best parameters found.

In [ ]:
#runtime= ~10 minutes

#early stopping
stop=EarlyStopping(monitor='val_loss', #quantity to be monitored
                    mode='min', #we look for decreasing patterns stop 
                    patience = 15, #number of epochs with no improvements
                    verbose = 1)
#data
features=np.shape(X_train)[1]
num_classes=10 
y_train_cat=to_categorical(y_train,num_classes)
y_val_cat=to_categorical(y_val,num_classes)


#model defining
np.random.seed(123)
set_random_seed=2

nn=Sequential()
nn.add(Dense(input_dim=features,
             units=features,
             activation='relu'))
nn.add(Dense(num_classes,
             activation='softmax'))
print(nn.summary())

#compiling and training
nn.compile(loss='categorical_crossentropy',
           optimizer='sgd',
           metrics=['accuracy'])
train=nn.fit(X_train_mm,y_train_cat,
             batch_size=32,
             verbose=0,
             callbacks=stop,
             epochs=100,
             validation_split=0.1)


In [ ]:
#display the accuracy 
plt.plot(train.history['accuracy'])
plt.plot(train.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation split'],loc='down right',fontsize='x-large')
plt.show()

#make prediction on training and validation set
y_pred_nn_train=nn.predict(X_train_mm).argmax(axis = 1) 
y_pred_nn_val=nn.predict(X_val_mm).argmax(axis = 1) 
print('minmax')
print(f'acc on train set: {accuracy_score(y_train,y_pred_nn_train):.5f}, acc on validation set:{accuracy_score(y_val,y_pred_nn_val):.5f}')


#expected output:
#minmax
#acc on train set: 0.94765, acc on validation set:0.89083


## What happen with Standard Scaler?

Now I another ANN using the data preprocessed by the standard scaler. The accuracy score reached on the validation is a little better then the one reached before.

In [ ]:
#early stopping
stop=EarlyStopping(monitor='val_loss', #quantity to be monitored
                    mode='min', #we look for decreasing patterns stop 
                    patience = 15, #number of epochs with no improvements
                    verbose = 1)

#model defining
np.random.seed(123)
set_random_seed=2

nns=Sequential()
nns.add(Dense(input_dim=features,
             units=features,
             activation='relu'))
nns.add(Dense(num_classes,
             activation='softmax'))
print(nns.summary())

#compiling and training
nns.compile(loss='categorical_crossentropy',
           optimizer='sgd',
           metrics=['accuracy'])
train_s=nns.fit(X_train_ss,y_train_cat,
             batch_size=16,
             verbose=0,
             callbacks=stop,
             epochs=100,
             validation_split=0.1)


#display the accuracy 
plt.plot(train_s.history['accuracy'])
plt.plot(train_s.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation split'],loc='down right',fontsize='x-large')
plt.show()

#make prediction on training and validation set
y_pred_train_ss=nns.predict(X_train_ss).argmax(axis = 1) 
y_pred_val_ss=nns.predict(X_val_ss).argmax(axis = 1) 
print('Standard Scaler')
print(f'acc on train set: {accuracy_score(y_train,y_pred_train_ss):.5f}, acc on validation set:{accuracy_score(y_val,y_pred_val_ss):.5f}')


#expected output:
#Standard Scaler
#acc on train set: 0.97587, acc on validation set:0.89417

# Convolutional Neural Network (Pytorch)

In [ ]:
# Pytorch import
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset, Subset
from torchvision import transforms, datasets
from training import train_epochs_acc

import matplotlib.pyplot as plt
import numpy as np

## Dataset and Dataloaders

In [ ]:
### Dataset

dataset = datasets.FashionMNIST('MNIST', train = True, download = True,
                             transform = transforms.Compose([
                               transforms.ToTensor(),
                                transforms.Normalize(0.0, 1.0, inplace=True),
                                 transforms.RandomAffine(10, (0.1,0.1), (0.95,1)),
                                 transforms.RandomCrop((28,28))
                             ]))

test_data = datasets.FashionMNIST('MNIST', train = False, download = True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                                 transforms.Normalize(0.0, 1.0, inplace=True)
                             ]))

# Divide data dataset in train dataset and val dataset
train_data, val_data = torch.utils.data.random_split(dataset, [50000, 10000])

In [ ]:
### Dataloaders
batch_size = 128
train_dataloader = DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers=0)
val_dataloader = DataLoader(val_data, batch_size = batch_size, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size = 1, shuffle=True, num_workers=0)

In [ ]:
# Plot some samples
figure = plt.figure(figsize=(8, 8))
cols, rows = 4,4


for i in range(1,cols * rows+1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Define the model

In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self, activation, drop_p):
        """
        The input is typically a MNIST images batch, encoded in a torch.tensor of size (N,1,28,28), where N is the batch size
        -----------
        Parameters:
        act = activation function 
        drop_p = dropout probability
        """
        super().__init__()
        
        # Parameters 
        self.act = getattr(nn, activation)(inplace = True)
        self.drop_p = drop_p
        
        
        ## Network architecture
        # Convolution part
        self.cnn = nn.Sequential(
            #first convolution layer
            nn.Conv2d(1, 16, 5),  # out = (N, 16, 24, 24)
            nn.BatchNorm2d(16),
            self.act,
            nn.Dropout(self.drop_p, inplace = False),
            nn.MaxPool2d(2),  # out = (N, 16, 12, 12)
            # Second convolution layer
            nn.Conv2d(16, 32, 5), # out = (N, 32, 8, 8)
            nn.BatchNorm2d(32),
            self.act,
            nn.Dropout(self.drop_p, inplace = False),
            nn.MaxPool2d(2) # out = (N, 32, 4, 4)
        )
    
        # Linear classifier
        self.lin = nn.Sequential(
            nn.Linear(in_features = 32*4*4, out_features = 128),
            nn.BatchNorm1d(128),
            self.act,
            nn.Dropout(self.drop_p, inplace = False),
            nn.Linear(in_features = 128, out_features = 10)
        )

        print("CNN initialized")
        
    def forward(self, x):
        # Convolution layer
        x = self.cnn(x)
        # Flatten layer
        x = torch.flatten(x, start_dim = 1)
        # Linear layer
        x = self.lin(x)
        return x
        
    

## Training

In [ ]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

In [ ]:
# Inizialize the network
torch.manual_seed(0)
net = ConvNet(activation="ReLU", drop_p=0.3)
net.to(device)

In [ ]:
# Define the loss function
loss_function = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr = 0.001)

# Training 
max_num_epochs = 1
early_stopping = True
train_loss_adam, val_loss_adam, accuracy_adam = train_epochs_acc(net, device, train_dataloader, val_dataloader, test_dataloader, loss_function, optimizer, max_num_epochs, early_stopping = early_stopping)


In [ ]:
### Plot losses
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(121)
ax1.semilogy(train_loss_adam, label='Train loss')
ax1.semilogy(val_loss_adam, label='Validation loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid()
ax1.legend()

ax2 = fig.add_subplot(122)
ax2.semilogy(accuracy_adam, label = "Test accuracy")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Test accuracy (%)")

plt.tight_layout()
plt.show()

In [ ]:
#k values tested
k_values= [2,4,8,10,16,32] 


#minmax scaler data
val_accuracy_mm = []

for k in k_values:
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(X_train_mms, y_train_small)
  y_pred_val = model.predict(X_val_mms)
  val_accuracy_mm.append(accuracy_score(y_val_small,y_pred_val))



#standard scaler data
val_accuracy_ss=[]

for k in k_values:
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(X_train_sss, y_train_small)
  y_pred_val_sss = model.predict(X_val_sss)
  val_accuracy_ss.append(accuracy_score(y_val_small,y_pred_val_sss))



#hog trnasformed data
val_accuracy_hog = []

for k in k_values:
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(X_train_hog, y_train_small)
  y_pred_val_hog = model.predict(X_val_hog)
  val_accuracy_hog.append(accuracy_score(y_val_small,y_pred_val_hog))



#accuracy on the small validation set
fig = plt.figure(figsize=(10,8))
plt.plot(k_values, val_accuracy_mm, label="val acc minmax")
plt.plot(k_values, val_accuracy_ss, label="val acc StandScal")
plt.plot(k_values, val_accuracy_hog, label="val acc Hog")
plt.xlabel("K")
plt.ylabel("Accuracy")  
plt.title('Accuracy score plot')
plt.legend()
plt.show()



model_mm = KNeighborsClassifier(n_neighbors=4)
model_mm.fit(X_train_mm, y_train)
y_pred_train_mm = model_mm.predict(X_train_mm)
y_pred_val_mm = model_mm.predict(X_val_mm)
print('Minmax scaled data')
print(f"accuracy score on train:{accuracy_score(y_train,y_pred_train_mm):.5}, on validation:{accuracy_score(y_val,y_pred_val_mm):.5f}")

model_ss = KNeighborsClassifier(n_neighbors=4)
model_ss.fit(X_train_ss, y_train)
y_pred_train_ss = model_ss.predict(X_train_ss)
y_pred_val_ss = model_ss.predict(X_val_ss)
print('Standard scaled data')
print(f"accuracy score on train:{accuracy_score(y_train,y_pred_train_ss):.5}, on validation:{accuracy_score(y_val,y_pred_val_ss):.5f}")

model_hog = KNeighborsClassifier(n_neighbors=4)
model_hog.fit(X_train_hog, y_train_small)
y_pred_train_hog = model_hog.predict(X_train_hog)
y_pred_val_hog = model_hog.predict(X_val_hog)
print('HOG transformed data')
print(f"accuracy score on train:{accuracy_score(y_train_small,y_pred_train_hog):.5}, on validation:{accuracy_score(y_val_small,y_pred_val_hog):.5f}")


#expected output:
#Minmax scaled data
#accuracy score on train:0.907, on validation:0.85833
#Standard scaled data
#accuracy score on train:0.9093, on validation:0.85750
#HOG transformed data
#accuracy score on train:0.8813, on validation:0.81667

# Conclusion

I can finally watch the performance on the test set using the best model found: the artificial neural network (wich works a little better then the SVM). 
Before this I only need to preprocess the Test Set using MinMax Scaler.

In [ ]:
#run time of final cell: 10minutes more or less

import tensorflow
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.utils import to_categorical 
from keras.utils.vis_utils import plot_model
from tensorflow.python.framework.random_seed import set_random_seed
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

#preprocessing
X_train_val_mm=X_train_val/255
X_test_mm=X_test/255
y_train_val_cat=to_categorical(Y_train_val,10)

#early stopping
stop=EarlyStopping(monitor='val_loss', #quantity to be monitored
                    mode='min', #we look for decreasing patterns stop 
                    patience = 15, #number of epochs with no improvements
                    verbose = 1)

#model defining
np.random.seed(123)
set_random_seed=2

nn_final=Sequential()
nn_final.add(Dense(input_dim=features,
             units=features,
             activation='relu'))
nn_final.add(Dense(num_classes,
             activation='softmax'))
print(nn_final.summary())

#compiling and training
nn_final.compile(loss='categorical_crossentropy',
           optimizer='sgd',
           metrics=['accuracy'])
train_final=nn_final.fit(X_train_val_mm,y_train_val_cat,
             batch_size=16,
             verbose=0,
             callbacks=stop,
             epochs=100,
             validation_split=0.1)


#display the accuracy and the loss
fig = plt.figure(figsize=(7,4))
plt.plot(train_final.history['accuracy'])
plt.plot(train_final.history['val_accuracy'])
plt.plot(train_final.history['loss'])
plt.plot(train_final.history['val_loss'])
plt.title('model accuracy & loss')
plt.ylabel('accuracy / loss')
plt.xlabel('epoch')
plt.legend(['train accuracy','val accuracy','train loss','val loss'],loc='down right',fontsize='x-large')
plt.show()


#make prediction on training and validation set
y_pred_test_final=nn_final.predict(X_test_mm).argmax(axis = 1) 
print('Accuracy')
print(f'test set: {accuracy_score(Y_test,y_pred_test_final):.5f}')


#expected output:
#Accuracy
#test set: 0.89100

The total run time of the notebook is 1 hour and 30 minutes